# Introduction
this notebook accompanies the work "The Growing N-Gram Algorithm: A Novel Approach to String Clustering" by Corrado Grappiolo, Eline Verwielen and Nils Noorman, in Proceedings of the 8th International Conference on Pattern Recognition Applications and Methods (ICPRAM19, http://www.icpram.org/)

# Imports

In [ ]:
import pandas as pd
import numpy as np
from nGram_recursiveKneserNey import recursive_NGramKneserNey as KN
import matplotlib.pyplot as plt
from scipy.stats import entropy

# Dataset Loading

In [ ]:
dataset = pd.read_csv("../data./xRaySequenceDataset.csv")

In [ ]:
dataset.head()

# Dataset statistics

In [ ]:
labels = ['train', 'test', 'validation', 'total']

In [ ]:
datasetDescr = {l : {} for l in labels}

In [ ]:
for l in labels :
    
    if l == 'total' :
        df = dataset.copy()
    else :
        df = dataset[dataset['type'] == l].copy()
    
    datasetDescr[l]['Size'] = len(df)
    datasetDescr[l]['Unique Strings'] = len(df['sequence'].drop_duplicates())
    
    if l not in ['train', 'total'] :
        datasetDescr[l]['String Intersection with Training Set'] = len(set(dataset[dataset['type'] == 'train']['sequence']).intersection(set(df['sequence'])))
    
    df['strLen'] = df['sequence'].apply(lambda x : len(x.split('-')))
    symbols = [x.split('-') for x in list(df['sequence'])]
    alphabeth = set([item for sublist in symbols for item in sublist])
    
    datasetDescr[l]['Alphabeth Size'] = len(alphabeth)
    
    if l not in ['train', 'total'] :
        df_train = dataset[dataset['type'] == 'train']
        symbols_train = [x.split('-') for x in list(df_train['sequence'])]
        alphabeth_train = set([item for sublist in symbols_train for item in sublist])
        datasetDescr[l]['Alphabeth Intersection with Training Set'] = len(alphabeth.intersection(alphabeth_train))
    
    datasetDescr[l]['Avg. String Length'] = np.round(np.mean(df['strLen']), 2)
    datasetDescr[l]['Std. Dev. String Length'] = np.round(np.std(df['strLen']), 2)
    datasetDescr[l]['Median String Length'] = np.round(np.median(df['strLen']), 2)
    datasetDescr[l]['Min. String Length'] = np.round(np.min(df['strLen']), 2)
    datasetDescr[l]['Max. String Length'] = np.round(np.max(df['strLen']), 2)
    
    #break

In [ ]:
for l in labels :
    print(l)
    for k in datasetDescr[l] :
        print('\t', k, datasetDescr[l][k])

# Growing N-Grams Algorithm Functions

In [ ]:
def growing_n_gram_algorithm(inputTrainingSet, 
                             tau_c_init, tau_u_init,
                             tau_c_delta = 1, tau_u_delta = 0.5,
                             tau_c_min = 1, tau_u_max = 1,
                             nGramValue = 2, dataColumnName = 'sequence',
                             printProgress = False) :
    
    trainingSet = inputTrainingSet.copy()
    trainingSet['fedToModels'] = False
    trainingSet.reset_index(inplace=True, drop=True)
    
    tau_c = tau_c_init
    tau_u = tau_u_init
    
    nGramDict = {}
    
    loopIndex = 0
    loop = True
    
    if printProgress :
        print("start growing ngram settings:")
        print("    tau_c_init:", tau_c_init)
        print("    tau_c_delta:", tau_c_delta)
        print("    tau_c_min:", tau_c_min)
        print("    tau_u_init:", tau_u_init)
        print("    tau_u_delta:", tau_u_delta)
        print("    tau_u_max:", tau_u_max)
        print("    n:", nGramValue)
      
    while loop :
        
        trainingSet = trainingSet[trainingSet['fedToModels'] == False]
        if len(trainingSet) == 0 :
            loop = False
            continue
        trainingSet = trainingSet.sample(frac = 1)
        trainingSet.reset_index(inplace=True, drop=True)

        nFed = 0
        nSkip = 0
        nCreate = 0
        nUpdate = 0

        if printProgress :
            print("loop =", loopIndex, 
                  ", toFeed =", len(trainingSet), 
                  ", tau_create =", tau_c, 
                  ", tau_update =", tau_u,
                  ", ensemble size =", len(nGramDict)) 

        for x in range(len(trainingSet)) :
            inputString = trainingSet.iloc[x][dataColumnName]

            inputStringSet = set(inputString.split('-'))

            M_c = []
            M_u = []

            for m in nGramDict :
                model_unigrams = set([k[0] for k in nGramDict[m].grams[1]])
                if len(inputStringSet.difference(model_unigrams)) > tau_c :
                    M_c.append(m)

                if len(inputStringSet.difference(model_unigrams)) <= tau_u :
                    M_u.append(m)

            if len(M_c) == len(nGramDict) :
                # creation
                newNgramIndex = len(nGramDict)
                nGramDict[newNgramIndex] = KN("ID_" + str(newNgramIndex) + "_n_" + str(nGramValue), 
                                              n=nGramValue, 
                                              useFixedVocabularySize=True, 
                                              fullVocabularySize=datasetDescr['total']['Alphabeth Size'] + 3,
                                              modelOOVfromTrainingData=False, 
                                              printTodo=False)

                nGramDict[newNgramIndex].feed(inputString)
                trainingSet.at[x, 'fedToModels'] = True
                nFed += 1
                nCreate += 1

            elif len(M_u) > 0 :
                # update
                probs = [nGramDict[m].chainProbability(inputString) for m in M_u]
                winnerModel = M_u[np.argmax(probs)]
                nGramDict[winnerModel].feed(inputString)
                trainingSet.at[x, 'fedToModels'] =  True
                nFed += 1
                nUpdate += 1

            else :
                # just skip the inputString, hence do nothing, do not update 'fedToModels
                nSkip += 1 


        # end of feeding pass, parameter update
        tau_u += tau_u_delta
        if tau_u > tau_u_max :
            tau_u = tau_u_max

        tau_c -= tau_c_delta
        if tau_c < tau_c_min :
            tau_c = tau_c_min

        loopIndex += 1
    
    if printProgress :
        print("loop =", loopIndex, 
              ", toFeed =", len(trainingSet), 
              ", tau_create =", tau_c, 
              ", tau_update =", tau_u,
              ", ensemble size =", len(nGramDict))
            
    return nGramDict

# Growing N-Grams Algorithm Execution, Entropy Calculation, Statistics

In [ ]:
trainingSet = dataset[dataset['type'] == 'train']
validationSet = dataset[dataset['type'] == 'validation']

In [ ]:
nRuns = 30

In [ ]:
ensembles = {}

In [ ]:
for r in range(nRuns) :
    # GNG algorithm execution
    print("start GNG run ", r)
    nGramDict = growing_n_gram_algorithm(trainingSet, 20, -1, printProgress=True)
    ensembles[r] = {'ensemble' : nGramDict}
    
    # entropy calculation on the validation set
    ensembleH = []
    for i in range(len(validationSet)) :
        x = validationSet.iloc[i]['sequence']
        chainProbs = [nGramDict[k].chainProbability(x) for k in nGramDict]
        ensembleH.append(entropy(chainProbs))
    ensembles[r]['validation_H'] = ensembleH
    print("Avg. Validation Entropy:", np.mean(ensembles[r]['validation_H']))
    print("Std. Validation Entropy:", np.std(ensembles[r]['validation_H']))
    print(20*'-')

In [ ]:
globalAvg = np.mean([ensembles[r]['validation_H'] for r in range(nRuns)])
globalStd = np.std([ensembles[r]['validation_H'] for r in range(nRuns)])

In [ ]:
globalEnselmbleSizeAvg = np.mean([len(ensembles[r]['ensemble']) for r in range(nRuns)])
globalEnselmbleSizeStd = np.std([len(ensembles[r]['ensemble']) for r in range(nRuns)])

In [ ]:
print("Global Avg. Entropy:", np.round(globalAvg, 4))
print("Global Std. Entropy:", np.round(globalStd, 4))
print("Global Avg. Enselmble Size:", np.round(globalEnselmbleSizeAvg, 4))
print("Global Std. Ensemble Size:", np.round(globalEnselmbleSizeStd, 4))